<a href="https://colab.research.google.com/github/astorguy/learn_ngspice/blob/widgets/notebooks/rlc_widgets/rlc_widgets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up

In [ ]:
# Removes files and directories that may be there from previous runs
#   also gets rid of the ubiquitous "sample_data" directory
!rm -rf *

In [ ]:
!sudo apt-get update && sudo apt-get install -y ngspice # install ngspice
# !sudo apt-get install -y ngspice # install ngspice
%pip install py4spice # Install py4spice package from PyPI

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,297 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from types import SimpleNamespace

import py4spice
import ipywidgets as widgets
from ipywidgets import interactive

In [ ]:
def is_running_in_colab() -> bool:
    '''Check to see if running in Colab or as a Jupyter notebook'''
    try:
        import google.colab  # type: ignore
        return True
    except ImportError:
        return False

# Ngspice simulation of RLC

## Paths
There are paths and files to define. Create `netlist` and `results` subdirectories. Make sure threre is an empty transcript log file in `results` subdirectory.

In [ ]:
@dataclass(frozen=True)
class AppPaths:
    ngspice = Path("/usr/bin/ngspice")
    content = Path("/content")
    netlists = Path("/content/netlists")
    results = Path("/content/results")
    top = Path("/content/netlists/top.cir")
    sim_transcript = Path("/content/results/sim_transcript.log")

    def create_dirs(self):
        """Creates the directories if they don't exist."""
        self.netlists.mkdir(parents=True, exist_ok=True)
        self.results.mkdir(parents=True, exist_ok=True)

PATHS = AppPaths() # instance to use throughout program
PATHS.create_dirs()
PATHS.sim_transcript.write_text("") # reset (create empty) transcript

0

## Vectors
Vectors objects are used to define a set of circuit signals for simulation and display.

In [ ]:
@dataclass(frozen=True)
class AppVectors:
    all= py4spice.Vectors("all")
    out = py4spice.Vectors("out")
    vil_branch = py4spice.Vectors("vil#branch")
    vic_branch = py4spice.Vectors("vic#branch")
    currents = vil_branch + vic_branch
    I_capacitor = py4spice.Vectors("I_capacitor")
    I_inductor = py4spice.Vectors("I_inductor")
    currents_renamed = I_capacitor + I_inductor

VECS = AppVectors() # instance to trhoughout program

## Netlists snippets
Netlist objects used as snippets when constructing the top netlist for simulating

In [ ]:
netlists = SimpleNamespace() # create the namespace for the class

netlists.blankline = py4spice.Netlist("")
netlists.title = py4spice.Netlist("* my ngspice netlist")
netlists.end = py4spice.Netlist(".end")

### ***Function:*** Simulation process function
Prepare top netlist and simulate. We put this in a function so we can easily modify parameters and loop.

In [ ]:
def sim_process(params):
    ''' There are several steps to the simulation process:
    1. Create list of analyses to run
    2. Create the control section
    3. Create DUT netlist object
    4. Construct the top netlist from control, DUT, and snippets
    5. Prepare Ngspice command
    6. Run the Ngspice simulation
    7. Create SimResults objects from the raw Ngspice output
    8. Create Waveforms objects from the SimResults objects
    '''
    # 1. Create list of analyses to run
    # ---------------------------------
    # (in our case only one analysis)
    list_of_analyses = []  # start with an empty list

    # 1st analysis: operating point (in our case, we have only one analysis)
    tran1 = py4spice.Analyses(
        name="tran1",
        cmd_type="tran",
        cmd=f"tran 1e-9 {params.tend} uic",
        vector=VECS.all,
        results_loc=PATHS.results,
    )
    list_of_analyses.append(tran1)


    # 2. Create the control section
    # -----------------------------
    my_control = py4spice.Control()  # create 'my_control' object

    # add all the analyses defined above into the control section
    for analysis in list_of_analyses:
        my_control.insert_lines(analysis.lines_for_cntl())

    # convert control section into a netlist object
    netlists.control = py4spice.Netlist(str(my_control))


    # 3. Create DUT netlist object
    # ----------------------------
    netlists.L = py4spice.Netlist(f"l1 out lmid {params.L} ic={params.ic}")
    netlists.vil = py4spice.Netlist("vil lmid 0 dc 0")
    netlists.c = py4spice.Netlist(f"c1 out cmid {params.C}")
    netlists.vic = py4spice.Netlist("vic cmid 0 dc 0")
    netlists.R = py4spice.Netlist(f"r1 out 0 {params.R}")

    netlists.dut = (
    netlists.L
    + netlists.vil
    + netlists.c
    + netlists.vic
    + netlists.R
    )
    # print(netlists.dut)


    # 4. Construct the top netlist from control, DUT, and snippets
    # ------------------------------------------------------------
    # concatenate netlist snippets to construct top
    netlists.top = (
        netlists.title
        + netlists.blankline
        + netlists.dut
        + netlists.blankline
        + netlists.control
        + netlists.blankline
        + netlists.end
        + netlists.blankline
        )
    # write netlist to a file so ngspice can read it
    netlists.top.write_to_file(PATHS.top)
    # print(netlists.top)


    # 5. Prepare Ngspice command
    # --------------------------
    # prepare simulate object and simulate
    sim = py4spice.Simulate(
        ngspice_exe=PATHS.ngspice,
        netlist_filename=PATHS.top,
        transcript_filename=PATHS.sim_transcript,
        name="sim1",
        timeout=5,
    )
    # print(sim.ngspice_command)


    # 6. Run the Ngspice simulation
    # -----------------------------
    sim.run()


    # 7. Create SimResults objects from the raw Ngspice output
    # --------------------------------------------------------
    sim_results: list[py4spice.SimResults] = [
        py4spice.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
        for analysis in list_of_analyses
    ]


    # 8. Create Waveforms objects from the SimResults objects
    # -------------------------------------------------------
    out_waves = py4spice.Waveforms(
        sim_results[0].header, sim_results[0].data_plot
        )

    return out_waves

### ***Function:*** Prepare waveforms
Reduces the set of `py4spice.Waveforms` and renames waves to better descriptions.

In [ ]:
def prepare_waveforms(waves):
    waves.vec_subset(VECS.currents.list_out())
    waves.scaler(1.0, "vil#branch", "I_inductor")  # rename
    waves.scaler(1.0, "vic#branch", "I_capacitor")  # rename
    return

### ***Function:*** Generate & display plot
Uses py4spice.Plot

In [ ]:
def generate_plot(waves, params):
    import matplotlib.pyplot as plt # Import matplotlib for text annotations

    # prepare plot
    plot_data = waves.x_axis_and_sigs(VECS.currents_renamed.list_out())
    y_names = VECS.currents_renamed.list_out()

    # plot
    my_plt = py4spice.Plot("tr_plt", plot_data, y_names, PATHS.results)

    # add to plot
    my_plt.set_title("RLC Transient Simulation Results")
    my_plt.define_axes(("time", "sec", "linear"), ("amps", "A", "linear"))

    # Add parameter values to the plot as text
    # Get the current axes to place text relative to data coordinates
    # Or use figtext for relative figure coordinates if axes are tricky
    # For simplicity, using plt.figtext which places text relative to the figure.
    plt.figtext(0.7, 0.81, f"L: {params.L*1e6:.1f} uH", fontsize=10, ha='left')
    plt.figtext(0.7, 0.79, f"C: {params.C*1e9:.1f} nF", fontsize=10, ha='left')
    plt.figtext(0.7, 0.77, f"R: {params.R:.1f} Ohm", fontsize=10, ha='left')
    plt.figtext(0.7, 0.75, f"ic: {params.ic*1e3:.1f} mA", fontsize=10, ha='left')
    plt.figtext(0.7, 0.73, f"tend: {params.tend*1e6:.1f} uSec", fontsize=10, ha='left')

### ***Function:*** Simulate and Plot

In [ ]:
def sim_and_plot(params):
    out_waves = sim_process(params)
    prepare_waveforms(out_waves)
    generate_plot(out_waves, params) # Pass params to generate_plot

### ***Function:*** Interactive simulation and Plot
Called by the sliders. This function calls the `sim_and_plot` function.

In [ ]:
def interactive_simulation_and_plot(L_value, C_value, R_value, ic_value, tend_value):
    # Apply the multiplication factors
    L_scaled = L_value * 1e-6
    C_scaled = C_value * 1e-9
    R_scaled = R_value * 1
    ic_scaled = ic_value * 1e-3
    tend_scaled = tend_value * 1e-6

    temp_params = CircuitParams(L=L_scaled, C=C_scaled, R=R_scaled, ic=ic_scaled, tend=tend_scaled)
    sim_and_plot(temp_params)

## Circuit parameters
Define the circuit parameters we want to vary to see out the simulation will change. Give them a set of initial values.

In [ ]:
@dataclass
class CircuitParams:
    L: float
    C: float
    R: float
    ic: float
    tend: float = 5e-6

# initial values
params = CircuitParams(L=10e-6, C=10e-9, R=100, ic=300e-3, tend=5e-6)

## Interactive sliders
Iteractive sliders linked to the simulation function. This enables dynamic changes to the parameters, resimulating and ploting.

In [ ]:
interactive_plot_widget = interactive(
    interactive_simulation_and_plot,
    L_value=widgets.FloatSlider(
        min=1, max=100, step=1, value=params.L / 1e-6, description='iL (uH)'
    ),
    C_value=widgets.FloatSlider(
        min=1, max=100, step=1, value=params.C / 1e-9, description='iC (nF)'
    ),
    R_value=widgets.FloatSlider(
        min=1, max=500, step=1, value=params.R / 1, description='R (ohm)'
    ),
    ic_value=widgets.FloatSlider(
        min=1, max=1000, step=1, value=params.ic / 1e-3, description='ic initial (mA)'
    ),
    tend_value=widgets.FloatSlider(
        min=1, max=100, step=1, value=params.tend / 1e-6, description='tend (uSec)'
    )
)
display(interactive_plot_widget);

interactive(children=(FloatSlider(value=10.000000000000002, description='iL (uH)', min=1.0, step=1.0), FloatSl…